In [20]:
import os

from dotenv import load_dotenv; load_dotenv()
from sqlalchemy import create_engine
from pgvector.sqlalchemy import Vector
from sqlalchemy.orm import declarative_base, mapped_column, Session

In [21]:
user = os.getenv('POSTGRES_USER')
pwd = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_HOST')
db = os.getenv('POSTGRES_DB')

engine = create_engine(f'postgresql+psycopg2://{user}:{pwd}@{host}/{db}')

In [53]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import Integer, BigInteger, Column, Text, DateTime

class Base(DeclarativeBase):
    pass

class Document(Base):
    __tablename__ = 'documents-new'

    id = Column(BigInteger, primary_key=True)
    embeddings = Column(Vector(768))
    title = Column(Text)
    url = Column(Text)
    content = Column(Text)
    tokens = Column(Integer)
    date_indexed = Column(DateTime)

In [54]:
# Create the table
Base.metadata.create_all(engine)

In [55]:
import datetime

with Session(engine) as session:
	item = Document(
		embeddings=[0.5 for _ in range(768)],
		title='Hello',
		url='http://example.com',
		content='World',
		tokens=2,
		date_indexed=datetime.datetime.now()
	)
	session.add(item)
	session.commit()
	print(item.embeddings)
	print(session.query(Document).first().embeddings)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.

In [43]:
from sqlalchemy.sql.expression import select

with Session(engine) as session:
    result = session.scalars(select(Document).order_by(Document.embeddings.l2_distance([1 for _ in range(768)])).limit(5))
    result = result.fetchall()
    for item in result:
        print(item.content)

World
World
